## Stableswap pool-math

In [1]:
import json
import numpy as np
from decimal import Decimal

from stableswappy.cst.exchg import StableswapPoolMath

In [2]:
def tkn2dec(tkn_amt, decimals):
    return int(Decimal(str(tkn_amt))*Decimal(str(10**decimals)))

def rate_multiplier(decimals):
    return 10 ** (36 - decimals)

def format_balances(balances, decimals):
    return [f"{b//10**d:,}" for b, d in zip(balances, decimals)]

In [3]:
GWEI  = 10**18

DECIMALS_GWEI = 18
DECIMALS_DAI = 18
DECIMALS_USDC = 6
DECIMALS_USDT = 6
DECIMALS = [DECIMALS_DAI, DECIMALS_USDC, DECIMALS_USDT]

In [4]:
amt_dai = 79566307.559825807715868071
amt_usdc = 81345068.187939
amt_usdt = 55663250.772939

In [5]:
decimal_amts = [tkn2dec(amt_dai, DECIMALS_DAI),
           tkn2dec(amt_usdc, DECIMALS_USDC),
           tkn2dec(amt_usdt, DECIMALS_USDT)]

rates = [rate_multiplier(DECIMALS_DAI), 
         rate_multiplier(DECIMALS_USDC), 
         rate_multiplier(DECIMALS_USDT)]

test_pool = StableswapPoolMath(A = 2000, 
                           D = decimal_amts, 
                           n = len(rates),
                           rates = rates, 
                           fee = 1000000)

format_balances(test_pool.balances, DECIMALS)

['79,566,307', '81,345,068', '55,663,250']

#### USDC / DAI (Price)

In [6]:
test_pool.dydx(1, 0, use_fee=False)

0.9999896456022903

In [7]:
test_pool.dydx(0, 1, use_fee=False)

1.0000103545049244

#### USDC / USDT (Price)

In [8]:
test_pool.dydx(1, 2, use_fee=False)

0.9997863487559973

In [9]:
test_pool.dydx(2, 1, use_fee=False)

1.0002136969006115

#### DAI / USDT (Price)

In [10]:
test_pool.dydx(0, 2, use_fee=False)

0.9997967010486687

In [11]:
test_pool.dydx(2, 0, use_fee=False)

1.000203340290199

#### Swap USDC for Tether

In [12]:
i = 1  # USDC (coming in)
j = 2  # Tether (going out)
dx = 10**4 * 10**DECIMALS_USDC

dy, fee = test_pool.exchange(i, j, dx)
dy/10**DECIMALS_USDT, fee/10**DECIMALS_USDT

(9996.862748, 0.999786)

In [13]:
print(format_balances(test_pool.balances, DECIMALS))

['79,566,307', '81,355,068', '55,653,253']


#### Swap DAI for Tether

In [14]:
i = 0  # DAI
j = 2  # Tether
dx = 10**4 * 10**DECIMALS_DAI

dy, fee = test_pool.exchange(i, j, dx)
dy/10**DECIMALS_USDT, fee/10**DECIMALS_USDT

(9996.964923, 0.999796)

In [15]:
print(format_balances(test_pool.balances, DECIMALS))

['79,576,307', '81,355,068', '55,643,256']


#### Swap USDC for DAI

In [16]:
i = 1  # USDC
j = 0  # DAI
dx = 10**4 * 10**DECIMALS_USDC

dy, fee = test_pool.exchange(i, j, dx)
dy/10**DECIMALS_DAI, fee/10**DECIMALS_DAI

(9998.895898397388, 0.9999895887986187)

In [17]:
print(format_balances(test_pool.balances, DECIMALS))

['79,566,308', '81,365,068', '55,643,256']


#### Add DAI

In [18]:
amt_dai = 100000
amt_tkns = [tkn2dec(amt_dai, DECIMALS_DAI), 0, 0]
test_pool.add_liquidity(amt_tkns)

format_balances(test_pool.balances, DECIMALS)

['79,666,308', '81,365,068', '55,643,256']

#### Add USDC

In [19]:
amt_usdc = 250000
amt_tkns = [0, tkn2dec(amt_usdc, DECIMALS_USDC), 0]
test_pool.add_liquidity(amt_tkns)

format_balances(test_pool.balances, DECIMALS)

test_pool.add_liquidity(amt_tkns)

249971400696586377656324

#### Remove DAI

In [20]:
test_pool.tokens/10**DECIMALS_GWEI

217172960.61976177

In [21]:
amt_lp = 250000
amt_lp_dec = tkn2dec(amt_lp, DECIMALS_GWEI) 
dy = test_pool.calc_withdraw_one_coin(amt_lp_dec, 0, False)
dy/10**DECIMALS_DAI

250013.72583753208

In [22]:
test_pool.remove_liquidity_one_coin(amt_lp_dec, 0)
format_balances(test_pool.balances, DECIMALS)

['79,416,294', '81,865,068', '55,643,256']

In [23]:
test_pool.tokens/10**DECIMALS_GWEI+amt_lp

217172960.61976177

#### Remove USDT

In [24]:
test_pool.tokens/10**DECIMALS_GWEI

216922960.61976177

In [25]:
amt_lp = 250000
amt_lp_dec = tkn2dec(amt_lp, DECIMALS_GWEI) 
dy = test_pool.calc_withdraw_one_coin(amt_lp_dec, 2, False)
dy/10**DECIMALS_USDT

249962.533927

In [26]:
test_pool.remove_liquidity_one_coin(amt_lp_dec, 2)
format_balances(test_pool.balances, DECIMALS)

['79,416,294', '81,865,068', '55,393,294']

In [27]:
test_pool.tokens/10**DECIMALS_GWEI+amt_lp

216922960.61976177

#### Remove USDC

In [28]:
test_pool.tokens/10**DECIMALS_GWEI

216672960.61976177

In [29]:
amt_lp = 250000
amt_lp_dec = tkn2dec(amt_lp, DECIMALS_GWEI) 
dy, fee = test_pool.calc_withdraw_one_coin(amt_lp_dec, 1, True)
dy/10**DECIMALS_USDC

250005.56071

In [30]:
test_pool.remove_liquidity_one_coin(amt_lp_dec, 1)
format_balances(test_pool.balances, DECIMALS)

['79,416,294', '81,615,050', '55,393,294']